In [1]:
### 라이브러리
import pandas as pd
from tqdm import tqdm
import time

# HTML 및 XML 문서를 파싱하는데 사용하는 라이브러리
from bs4 import BeautifulSoup

# Selenium 모듈에서 웹 자동화 프레임워크에서 웹요소를 찾을때 사용하는 클래스
from selenium.webdriver.common.by import By

# Selenium 모듈에서 webdriver를 가져옵니다.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
 
# Chrome 브라우저 옵션을 다루는 Options 클래스를 가져옵니다.
from selenium.webdriver.chrome.options import Options
 
# Options 클래스의 인스턴스를 생성합니다.
options = Options()

# 페이지 로딩 시간을 줄이기 위해 이미지 로딩 비활성화
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

options.add_argument("--window-size=50,300")
options.add_argument("--window-position=15,15")

# 병렬처리
import concurrent.futures

In [2]:
df = pd.read_excel("./prod_url.xlsx")
df

,Unnamed: 0,L-카르니틴,MSM,N-아세틸글루코사민,UDCA,가르시니아캄보지아,감마리놀렌산(GLA),감마오리자놀,감태,곰피,...,프리바이오틱스,피크노제놀,허니부쉬추출물,헛개나무열매,헤모힘,홍경천추출물,홍국,홍삼,회화나무열매추출물,히알루론산
0,0,"['/products/1631/L-카르니틴-1000mg', '/products/70...","['/products/188/MSM-1500mg', '/products/9578/M...","['/products/6828/관절엔-MSM', '/products/6549/N-아...","['/products/7377/우루사-100mg', '/products/8564/우...","['/products/6887/(단종)-락토핏-생유산균-슬림', '/products...","['/products/117/감마리놀렌산-달맞이꽃종자유-1001mg', '/prod...","['/products/7211/마그비감마', '/products/15304/액티넘-...","['/products/7136/리얼-슬립', '/products/7416/감태잠태'...","['/products/23317/심플리간', '/products/7330/남심男心-...",...,['/products/12723/닥터포뮬레이티드-프로바이오틱스-원스-데일리-우먼-5...,"['/products/18347/피크노제놀-100mg', '/products/158...","['/products/4294/이너비-스노우-화이트', '/products/3254...","['/products/19231/간건강-간-피로-케어-쿠퍼스', '/products...",['/products/33/헤모힘'],"['/products/24342/콜레올로지-컷', '/products/7212/마이...","['/products/7532/에너씨슬-콜레다운', '/products/15984/...","['/products/3/홍삼정-에브리타임-10ml', '/products/1804...","['/products/5038/(단종)-르시크릿-에스트로벤', '/products/...","['/products/16038/트리플콜라겐-오렌지', '/products/2303..."


In [3]:
list(df.columns[21:31])

['루테인',
 '마그네슘',
 '마카',
 '망간',
 '매스틱검',
 '맥주효모',
 '몰리브덴',
 '뮤코다당･단백',
 '밀크씨슬',
 '밀크씨슬(실리마린)']

In [4]:
def getUrl(ntr):
    # 크롬 드라이버 실행
    ntr_driver = webdriver.Chrome()

    # 각 성분 웹페이지 접속
    ntr_url = f"https://www.***.com/search/nutrients?query={ntr}"
    ntr_driver.get(ntr_url)

    # 30개 더보기 버튼 있는 대로 누르기
    while True:
        try : 
            xPATH1 = '//*[@id="moreBtn"]'
            ntr_driver.find_element(By.XPATH,xPATH1).send_keys(Keys.ENTER)
            time.sleep(0.4)
        except:
            break

    # 제품명 url 파싱
    html = ntr_driver.page_source 
    ntr_soup = BeautifulSoup(html, 'lxml')
    ntr_driver.quit()

    # 제품명 url 리스트에 담기 
    prod_each_url = [v["href"] for v in ntr_soup.select("a.item-card")]
    return prod_each_url

In [5]:
######## [각 제품 별 고유 주소값을 인자로 받는 함수] ########
# - ntr 인자 예시 : L-카르니틴
# - idv_url 인자 예시 : /products/1631/L-카르니틴-1000mg

def getProdInfo(idv_url, ntr):
    prod_driver = webdriver.Chrome(options=options)
    prod_url = "https://www.***.com"+idv_url
    prod_driver.get(prod_url)

    # 제품명 url 파싱
    html = prod_driver.page_source 
    prod_soup = BeautifulSoup(html, 'lxml')
    prod_driver.quit()

    ### [제조사,제품명,별점,리뷰수,건기식여부, 기능, 추천섭취시간,추천복용법] 데이터 추출하기
    # 제조사
    prod_mnf = prod_soup.select("div.col > a.search-brand > span.txt1")
    # 제품명
    prod_nm = prod_soup.select("div.col h1.txt2")
    # 별점,리뷰수 -> 오류 시 없음으로 표기
    try :
        rv_scr = prod_soup.select("div.col div.review span.star-point")
        rv_cnt = prod_soup.select("div.col div.review span.amount")
    except:
        rv_scr = "없음"
        rv_cnt = "없음"
    # 건기식여부 
    try : prod_hff = prod_soup.select("div.col div.marks")
    except: pass
    # 주요기능
    main_func = prod_soup.select("#product-detail > div.section.section3-2  div.card-tags")[0].select("div")
    # 보조기능 -> 오류 시 없음으로 표기
    try : sub_func = prod_soup.select("#product-detail > div.section.section3-2  div.card-tags")[1].select("div")
    except : sub_func = "없음"
    # 추천섭취시간
    temp1 = prod_soup.select("#product-detail > div.section.section3-3 > div.card-wrap.intake > div > span.txt1")
    temp2 = prod_soup.select("#product-detail > div.section.section3-3 > div.card-wrap.intake > div > span.txt2")
    intk_time = [temp1[i].text+"|"+temp2[i].text for i in range(len(temp1))]
    # 추천복용법
    intk_how = prod_soup.select("div.section.section3-3 > div.card-wrap.card-wrap2 > span.txt3")

    # 데이터 프레임 만들기
    df_list = [ntr, prod_mnf, prod_nm, rv_scr, rv_cnt, prod_hff, main_func, sub_func,intk_time, intk_how]
    return df_list

In [ ]:
cols = ["prod_gb", "prod_mnf", "prod_nm", "rv_scr", "rv_cnt", "prod_hff", "main_func", "sub_func", "intk_time", "intk_how"]

def process_ntr(ntr):
    print(ntr)
    try:
        ### 영양성분별 제품url리스트 가져오기 - df에서
        temp_list = eval(df[ntr][0])
        
        # 제품 정보, 리스트로 저장 후 df로 변환
        list_temp = []
        for idv_url in tqdm(temp_list):
            try:
                list_temp += [getProdInfo(idv_url, ntr)]
            except:
                print("*** 오류 : " + idv_url)

        df_ntr = pd.DataFrame(list_temp, columns=cols)

        # csv 파일로 저장
        df_ntr.to_csv(f"{ntr}.csv")
        
    except:
        ### 영양성분별 제품url리스트 가져오기 - 새로 파싱
        temp_list = getUrl(ntr)
        
        # 제품 정보, 리스트로 저장 후 df로 변환
        list_temp = []
    
        # - temp_list를 500개씩 나누어 처리
        chunk_size = 500
        for i in range(0, len(temp_list), chunk_size):
            chunk = temp_list[i:i + chunk_size]
            for idv_url in tqdm(chunk):
                try:
                    list_temp += [getProdInfo(idv_url, ntr)]
                except:
                    print("*** 오류 : " + idv_url)
            # 각 부분을 별도의 CSV 파일로 저장
            df_ntr = pd.DataFrame(list_temp, columns=cols)
            df_ntr.to_csv(f"{ntr}_part{i // chunk_size}.csv")

if __name__ == '__main__':
    ######## 특정 성분으로 크롤링할 때, 아래의 코드 수치 변경 df.columns[*변경*] ########
    ntr_list = df.columns[21:31]

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        executor.map(process_ntr, ntr_list)

루테인
마그네슘
마카


  0%|▏                                                                              | 3/1028 [00:39<3:37:13, 12.72s/it]

  3%|██▏                                                                               | 5/188 [01:02<37:38, 12.34s/it]

  3%|██▌                                                                               | 6/188 [01:14<36:57, 12.18s/it]

  4%|███                                                                               | 7/188 [01:27<37:12, 12.34s/it]

  1%|▌                                                                              | 8/1028 [01:40<3:31:47, 12.46s/it]

  5%|███▉                                                                              | 9/188 [01:52<36:37, 12.28s/it]

  5%|████▎                                                                            | 10/188 [02:03<35:50, 12.08s/it]

  6%|████▋                                                                            | 11/188 [02:15<35:11, 11.93s/it]

  6%|█████▏                     

  7%|█████                                                                         | 67/1028 [13:12<3:09:00, 11.80s/it]

 37%|██████████████████████████████▏                                                  | 70/188 [13:13<23:00, 11.70s/it]

 38%|██████████████████████████████▌                                                  | 71/188 [13:25<23:17, 11.94s/it]

  7%|█████▏                                                                        | 69/1028 [13:38<3:18:12, 12.40s/it]

  7%|█████▎                                                                        | 70/1028 [13:50<3:15:19, 12.23s/it]

 39%|███████████████████████████████▉                                                 | 74/188 [14:01<22:43, 11.96s/it]

  7%|█████▍                                                                        | 72/1028 [14:13<3:12:03, 12.05s/it]

 40%|████████████████████████████████▋                                                | 76/188 [14:25<22:13, 11.91s/it]

  7%|█████▌                     

 72%|█████████████████████████████████████████████████████████▊                      | 136/188 [26:01<10:07, 11.67s/it]

 73%|██████████████████████████████████████████████████████████▎                     | 137/188 [26:13<09:56, 11.70s/it]

 73%|██████████████████████████████████████████████████████████▋                     | 138/188 [26:25<09:44, 11.68s/it]

 74%|███████████████████████████████████████████████████████████▏                    | 139/188 [26:36<09:32, 11.69s/it]

 74%|███████████████████████████████████████████████████████████▌                    | 140/188 [26:48<09:19, 11.66s/it]

 75%|████████████████████████████████████████████████████████████                    | 141/188 [27:03<09:53, 12.62s/it]

 13%|██████████▎                                                                  | 137/1028 [27:04<3:05:39, 12.50s/it]

 76%|████████████████████████████████████████████████████████████▊                   | 143/188 [27:26<08:58, 11.97s/it]

 77%|███████████████████████████

*** 오류 : /products/6435/눈건강엔-루테인-아스타잔틴


...

 88%|███████████████████████████████████████████████████████████████████▍         | 900/1028 [2:36:49<19:58,  9.36s/it]

 88%|███████████████████████████████████████████████████████████████████▍         | 901/1028 [2:36:59<19:47,  9.35s/it]

 88%|███████████████████████████████████████████████████████████████████▌         | 902/1028 [2:37:08<19:38,  9.35s/it]

 88%|███████████████████████████████████████████████████████████████████▋         | 903/1028 [2:37:17<19:32,  9.38s/it]

 88%|███████████████████████████████████████████████████████████████████▋         | 904/1028 [2:37:27<19:24,  9.39s/it]

 88%|███████████████████████████████████████████████████████████████████▊         | 905/1028 [2:37:36<19:17,  9.41s/it]

 88%|███████████████████████████████████████████████████████████████████▊         | 906/1028 [2:37:46<19:07,  9.41s/it]

 88%|███████████████████████████████████████████████████████████████████▉         | 907/1028 [2:37:55<18:54,  9.38s/it]

 88%|███████████████████████████

*** 오류 : /products/31863/플래티넘-루테인11



 89%|████████████████████████████████████████████████████████████████████▉        | 920/1028 [2:41:04<48:18, 26.83s/it]

*** 오류 : /products/2314/칼마디-플러스




 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                            | 416/500 [1:07:22<37:56, 27.11s/it]